Parositasi algoritmus Paros grafokban

A paros graf szomszedsagi matrix sorai betuvel oszlopai szamokkal vannak jelolve

In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
# from sage.graphs.graph_plot import GraphPlot
# from shapely.geometry import LineString
from copy import deepcopy
#from networkx.drawing.nx_pydot import graphviz_layout
from networkx.drawing.nx_agraph import graphviz_layout
from ipywidgets import interact
from ipywidgets.widgets import IntSlider
from matplotlib.ticker import AutoMinorLocator
from networkx.drawing.nx_agraph import graphviz_layout

In [2]:
A=[[1,0,0,0,0,0,1,0,1,1],[0,1,0,0,1,0,0,0,0,0],[0,0,1,0,1,1,0,1,1,0],[0,0,0,1,0,0,1,1,1,0],[1,0,0,1,1,0,0,0,0,0],[0,0,0,0,0,1,1,1,0,1],[0,1,1,1,0,0,0,0,0,0],[0,1,1,0,0,0,0,0,0,0],[0,1,1,0,0,0,0,0,0,0]]

d_inc={'a':[1,7,9,10],'b':[2,5],'c':[3,5,6,8,9],'d':[4,7,8,9],'e':[1,4,5],'f':[6,7,8,10],'g':[2,3,4],'h':[2,3],'i':[2,3]}

A_class=sorted(list(d_inc.keys()))
B_class=list(set([u for v in d_inc.keys() for u in d_inc[v]]))

G=nx.Graph(d_inc)

In [5]:
algoritmus = [
    u'Az aktuális párosítás, Javító út keresés indítása!',
    u'\t BFS-t indítunk az M-fedetlen A-beliekből', 
    u'\t B-beliek fertőznek minden M-menti szomszédot', 
    u'\t A-beliek fertőznek mindenkit, akit lehet', 
    u'Van-e javító út?', 
    u'\tVan javító út.',
    u'\t  Javítsuk M-et és Vissza az eléjere', 
    u'\tNincs javító út. Lefogó ponthalmaz kiszámítása!'
]

In [6]:
d_movie={}
szamlalo=0
Fontos_Index=[]

M=[]
eps=0.5
b_found_alternating_path=True

while(b_found_alternating_path):
    MG=nx.Graph(M)
    
    Fedett=[u for e in M for u in e]
    
    Fedetlen_A=[u for u in A_class if u not in Fedett]
    
    #BFS starting from Fedetlen_A
    Q=Fedetlen_A
    volt={u:False for u in A_class+B_class}
    tav={u:0 for u in A_class+B_class}
    honnan={u:'*' for u in A_class+B_class}
    
    d_seged={'matching':M,
             'selected_A':(False,[]),
             'selected_B':(False,[]),
             'segedgraf':(False,[]),
             'fertozottek':(False,[]),
             'alternalo_javito_ut':(False,[]),
             'lefogo_A':(False,[]),
             "lefogo_B":(False,[]),
             'code_index':0,
             'matching_torolni':(False,[]),
             'matching_ujak':(False,[])}
    d_movie[szamlalo]=deepcopy(d_seged)
    Fontos_Index.append(szamlalo)
    szamlalo+=1
    
    b_found_alternating_path=False
    for u in Q:
        volt[u]=True
    #print(Q)
    while((len(Q)>0) ):# and (b_found_alternating_path==False)):
        akt=Q.pop(0)
        
        if tav[akt]%2==0: #akt is in A
            l_seged=[u for u in G.neighbors(akt) if volt[u]==False]
            Q+=l_seged
            for u in l_seged:
                volt[u]=True
                tav[u]=tav[akt]+1
                honnan[u]=akt
        else:
            if akt in Fedett:
                l_seged=[u for u in MG.neighbors(akt)]
                Q+=l_seged
                for u in l_seged:
                    volt[u]=True
                    tav[u]=tav[akt]+1
                    honnan[u]=akt
            else:
                if b_found_alternating_path==False:
                    fedetlen_veg=akt
                b_found_alternating_path=True
                

    T=nx.DiGraph([(u,honnan[u]) for u in honnan if volt[u]==True])
    pos_T={}
    if len(T.nodes())>0:
        pos_T= {k:(v[1],-v[0]) for k,v in graphviz_layout(T,prog='dot',root=['*']).items()}
        T=nx.to_undirected(T)
        T = nx.Graph(T)
        T.remove_node('*')     
        del(pos_T["*"])
        
        boundaries={
            'xlim':(min([pos_T[u][0] for u in T.nodes()]),max([pos_T[u][0] for u in T.nodes()])),
            'ylim':(min([pos_T[u][1] for u in T.nodes()]),max([pos_T[u][1] for u in T.nodes()]))}
        
        T_coloring_edges1={
            'black': [e for e in T.edges() if min([tav[v] for v in e])%2==0 ], 
            'blue': [e for e in T.edges() if min([tav[v] for v in e])%2==1] }
        for j in range(max(tav[u] for u in T.nodes())+1):
            T2=deepcopy(T)
            L_f=[u for u in T.nodes() if tav[u]<j+1]
#             print(len(L_f))
            T2.remove_nodes_from([u for u in T.nodes() if u not in L_f])
            pos_T2={u:pos_T[u] for u in T2.nodes()}
            T2_coloring_edges={
                'black': [e for e in T2.edges() if min([tav[v] for v in e])%2==0 ], 
                'blue': [e for e in T2.edges() if min([tav[v] for v in e])%2==1] }
            
            d_seged['fertozottek']=(True,tuple(L_f))
            d_seged['segedgraf']=(True,{
                'graf':T2,
                'position':pos_T2,
                'thickness':3,
                'edge_color':T2_coloring_edges,
                'xlim':boundaries['xlim'],
                'ylim':boundaries['ylim']
            })
            if j%2==0:
                d_seged['selected_A']=(True,[u for u in T2.nodes() if tav[u]==j])
                d_seged['selected_B']=(False,[])
                if j==0:
                    d_seged['code_index']=1
                else:
                    d_seged['code_index']=3
            else:
                d_seged['selected_A']=(False,[])
                d_seged['selected_B']=(True,[u for u in T2.nodes() if tav[u]==j])
                d_seged['code_index']=2
            d_movie[szamlalo]=deepcopy(d_seged)
            szamlalo+=1
        d_seged['selected_A']=(False,[])
        d_seged['selected_B']=(False,[])
        d_seged['code_index']=4
        d_movie[szamlalo]=deepcopy(d_seged)
        szamlalo+=1
        Fontos_Index.append(szamlalo)

        
    if b_found_alternating_path: #we found
        #plt.figure()
        #T=nx.Graph([(u,honnan[u]) for u in honnan if volt[u]==True])
        #pos = graphviz_layout(T, prog="dot",root='*')
        #print(list(T.nodes))
        #nx.draw_networkx(T, pos)

        #print(T.edges(labels=False,sort=False))
        #T.show(pos=pos_d)
        
        # GRAPH PLOTTING!!!
        #T.show(pos=pos_T,edge_colors=T_coloring_edges1,edge_thickness=3)
        
        L=[fedetlen_veg]
        akt=fedetlen_veg
        while honnan[akt]!='*':
            akt=honnan[akt]
            L.append(akt)
        L.reverse()
        
        if len(T.nodes())>0:
            L_pv=[pos_T[u] for u in L]
            d_seged['alternalo_javito_ut']=(True,L_pv)
            d_seged['code_index']=5
            d_movie[szamlalo]=deepcopy(d_seged)
            Fontos_Index.append(szamlalo)
            szamlalo+=1
            
#             C=line([(a,b) for a,b in L_pv],color='red',thickness=10,alpha=0.6)
            
            #C+=line([(a,b-eps) for a,b in L_pv],color='orange')
            #C+=circle(L_pv[0],eps, color='orange')
            #C+=circle(L_pv[-1],eps, color='orange')
#             C+=T.plot(pos=pos_T,edge_colors=T_coloring_edges1,edge_thickness=3)
#             C.show(axes=False)

        #T_coloring_edges2={'red':[]}
        
#         d_seged['alternalo_javito_ut']=(True,L)
        
        M_torolni=[(L[i+1],L[i]) for i in range(1,len(L),2) if i+1<len(L)]
        M_uj=[(L[i],L[i+1]) for i in range(0,len(L),2) if i+1<len(L)]
        
        d_seged['matching_torolni']=(True,M_torolni)
        d_seged['matching_ujak']=(True,M_uj)
        d_seged['code_index']=6
        d_movie[szamlalo]=deepcopy(d_seged)
        szamlalo+=1
        Fontos_Index.append(szamlalo)
        
        M=[e for e in M if e not in M_torolni]+M_uj
    else:
        Lefogo_A=[u for u in A_class if volt[u]==False]
        Lefogo_B=[u for u in B_class if volt[u]==True]
        Lefogo=Lefogo_A+Lefogo_B
        d_seged['lefogo_A']=(True,Lefogo_A)
        d_seged['lefogo_B']=(True,Lefogo_B)
        d_movie[szamlalo]=deepcopy(d_seged)
        d_seged['code_index']=7
        Fontos_Index.append(szamlalo)
        szamlalo+=1

# print(M)
# print(len(M))
# print(len(Lefogo))

# L=[(u,(i,0)) for i, u in enumerate(A_class)]+[(u,(i,1)) for i, u in enumerate(B_class)]
# pos_G={u:p for u,p in L}
# sG=Graph(G)
# G_coloring={'red':[u for u in sG.vertices(sort=False) if u in Lefogo], 'blue':[u for u in sG.vertices(sort=False) if u not in Lefogo]}
# sG.show(pos=pos_G,figsize=(10,10),vertex_colors=G_coloring)

ImportError: ('requires pygraphviz ', 'http://pygraphviz.github.io/')

In [7]:
def plot_all(index):
    data = d_movie[index]
    
#     fig,ax = plt.subplots(1,4,figsize=(20,5))
    
    fig = plt.figure(figsize=(15,12))
    gs = fig.add_gridspec(2, 3,hspace=0.01)
    
    ax1 = fig.add_subplot(gs[0, 0])
    ax0 = fig.add_subplot(gs[0, 1:3])
    ax2 = fig.add_subplot(gs[1, 0:2])
    ax3 = fig.add_subplot(gs[1, 2])
    ax = [ax0,ax1,ax2,ax3]
    
    # FIG 0
        
    for i,line in enumerate(algoritmus):
        tab_offset=0.1
        dy=0.07
        starting_height=(len(A_class)+1)*dy
        if i!=data["code_index"]:
            if line[0]=="\t":
                ax[0].annotate(line[1:],(0+tab_offset,starting_height-dy*i),fontsize=12)
            else:
                ax[0].annotate(line,(0,starting_height-dy*i),fontsize=12)
        else:
            if line[0]=="\t":
                ax[0].annotate(line[1:],(0+tab_offset,starting_height-dy*i),fontsize=12,weight="bold")
            else:
                ax[0].annotate(line,(0,starting_height-dy*i),fontsize=12,weight="bold")
    ax[0].set_axis_off()
    
    # FIG 1
    # adjacency matrix
    ax[1].imshow(A,interpolation="nearest",cmap="Greys")
    ax[1].xaxis.tick_top()
    ax[1].set_xticks([e for e in range(len(B_class))])
    ax[1].set_xticklabels(B_class,fontdict={"fontsize":15})
    ax[1].set_yticks([e for e in range(len(A_class))])
    ax[1].set_yticklabels(A_class,fontdict={"fontsize":15})
    ax[1].minorticks_on()
    ax[1].set_ylabel("A osztály",fontdict={"fontsize":15})
    ax[1].set_xlabel("B osztály",fontdict={"fontsize":15},labelpad=10)
    ax[1].xaxis.set_label_position('top') 

    # placing 1 minor tick between 2 major ticks
    aml = AutoMinorLocator(2)
    ax[1].xaxis.set_minor_locator(aml)
    ax[1].yaxis.set_minor_locator(aml)
    ax[1].xaxis.set_tick_params(which='both', length=0,pad=7)
    ax[1].yaxis.set_tick_params(which='both', length=0,pad=7)
    ax[1].grid(which="minor")

    # adding colored circles for a certain matching
    for edge in data["matching"]:
        ax[1].plot(B_class.index(edge[1]),A_class.index(edge[0]),'ro',markersize=14)

    # adding colored lines for a certain covering
    A_lefogo = ['a','b','e']
    B_lefogo = [4,7,8]

    if data["lefogo_A"][0]:
        for elem in data["lefogo_A"][1]:
            ax[1].plot([0-0.5,len(B_class)-0.5],[A_class.index(elem),A_class.index(elem)],'b-',lw=3)
    if data["lefogo_B"][0]:
        for elem in data["lefogo_B"][1]:
            ax[1].plot([B_class.index(elem),B_class.index(elem)],[0-0.5,len(A_class)-0.5],'b-',lw=3)

    # highlighting tick labels for selected nodes (in both classes)
    if data["selected_A"][0]:
        for elem in data["selected_A"][1]:
            ax[1].get_yticklabels()[A_class.index(elem)].set_color("red")
    if data["selected_B"][0]:
        for elem in data["selected_B"][1]:
            ax[1].get_xticklabels()[B_class.index(elem)].set_color("red")
            
    ax[1].set_aspect(1)
            
    # FIG 2
    
    if data["alternalo_javito_ut"][0]:
        x = [elem[0] for elem in data["alternalo_javito_ut"][1]]
        y = [elem[1] for elem in data["alternalo_javito_ut"][1]]
        ax[2].plot(x,y,'-',lw=15,color='orange',alpha=0.5)
        
    if data["segedgraf"][0]:
        G_info = data["segedgraf"][1]
        T = G_info["graf"]
        nx.draw_networkx_nodes(T,G_info["position"],ax=ax[2],node_size=400)
        for color in G_info["edge_color"]:
            nx.draw_networkx_edges(
                T,
                G_info["position"],
                ax=ax[2],
                edgelist=G_info["edge_color"][color],
                edge_color='black',
                width=6
            )
            if color=="black":
                nx.draw_networkx_edges(
                    T,
                    G_info["position"],
                    ax=ax[2],
                    edgelist=G_info["edge_color"][color],
                    edge_color='white',
                    width=3
                )
        nx.draw_networkx_labels(T,G_info["position"],ax=ax[2])
        xpadding=(G_info["xlim"][1]-G_info["xlim"][0])*0.15
        ypadding=(G_info["ylim"][1]-G_info["ylim"][0])*0.15
        ax[2].set_xlim(G_info["xlim"][0]-xpadding,G_info["xlim"][1]+xpadding)
        ax[2].set_ylim(G_info["ylim"][0]-ypadding,G_info["ylim"][1]+ypadding)
        ax[2].set_aspect(np.sqrt((G_info["xlim"][1]-G_info["xlim"][0])/(G_info["ylim"][1]-G_info["ylim"][0])))
    ax[2].set_axis_off()
 
    
    # FIG 3
    
    ax[3].annotate(u"Fertőzhető csúcsok:",(0,0.7),fontsize=15)
    ax[3].annotate("A osztály: ",(0.1,0.6),fontsize=15)
    ax[3].annotate("B osztály: ",(0.1,0.5),fontsize=15)
    
    if data["fertozottek"][0]:
        dx=0.07

        for i,elem in enumerate(A_class):
            if elem not in data["fertozottek"][1]:
                ax[3].annotate(str(elem),(0.5+i*dx,0.6),fontsize=15)
        
        for i,elem in enumerate(B_class):
            if elem not in data["fertozottek"][1]:
                ax[3].annotate(str(elem),(0.5+i*dx,0.5),fontsize=15)


    ax[3].set_axis_off()

In [8]:
interact(plot_all,index=IntSlider(0,0,len(d_movie.keys())-1,1))

interactive(children=(IntSlider(value=0, description='index', max=0), Output()), _dom_classes=('widget-interac…

<function __main__.plot_all(index)>

In [9]:
d_movie[78]

KeyError: 78